# A very basic project management system

Who doesn't need that? Everyone needs something like that at some point in time. 

So how do you build one?

----------

Let's start with the nomenclature:

**Projects** are directed acyclic graphs (DAGs).

DAGs are made of links (indicating dependencies) & nodes (**tasks**).


**Tasks & Dependencies** require resources in the form of materials and skills. Skills determine the time for the task to be completed.


**Tasks** ...
1. have mutually exclusive options. Do task A and B is not required. Dependending on which path is taken, the projects overall completion time may change.
2. are repeated. The dependencies allow these to be listed sequentially.
3. have deadlines and fines are due if they're exceeded.

Projects have a critical path that determines the shortest possible time to completion. 


**Slack**: Tasks that are not on the critical path have "slack" which is the excess time before or after that leads permits earlier or later start(s) without influencing the critical path.


**Programmes** programmes are containers of 2 or more projects that must share resources.
Multiple programmes do not interwene in one-another. 


At this point we can create programmes, projects and tasks. 
The next item is then to assure that a cascade of timedeltas are possible, so that the critical path can be calculated.

The inconvenient truth is that durations may depend on "who" does the task, or more generally, which resources are assigned to the task.
This creates a matrix between tasks and resources, and subsequently demands the search for combinations that avoids duplicated assignments.

Assuming we have tasks 1,2,3 & 4 and the resources Alice, Bob & Charlie, then we can evaluate how much time each task takes.

Let's start with asking ourselves in how many ways Alice, Bob & Charlie can work together?

In [20]:
from itertools import chain, combinations

def powerset(iterable, min_=0):
    "powerset([1,2,3]) --> (), (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(min_, len(s)+1))

Next we ask each of them how much time a task takes:

In [15]:
tasks = [1,2,3,4]
estimates = {
    "alice" : [10,11,9,12],  # hours!
    "bob" : [11,12,11,11],
    "charlie" : [9,10,9,9]
}

With assumption is that Alice completes 1/10 task (rate) in the same time that bob complete 1/11 task (also rate), we may estimate how much time would it take if they work together using the harmonic mean.

In [12]:
def inv_mean(values):
    """inv_mean([10,11]) --> 5.238 """
    return 1 / sum(1/n for n in values)

So the task may now be computed as:

In [26]:
team = []
for task_index, task in enumerate(tasks):
    for person, times in estimates.items():
        team.append( (task, times[task_index], (person, )))

for task_index, task in enumerate(tasks):
    for combination in powerset(["alice","bob","charlie"],2):  # (alice, bob)  --> ([10,11,9,12], [11,12,11,11])
        times = [estimates[person][task_index] for person in combination]
        time = inv_mean(times)
        team.append((task, time, combination))
        print(task, "|", round(time,4), "|", combination)

1 | 5.2381 | ('alice', 'bob')
1 | 4.7368 | ('alice', 'charlie')
1 | 4.95 | ('bob', 'charlie')
1 | 3.311 | ('alice', 'bob', 'charlie')
2 | 5.7391 | ('alice', 'bob')
2 | 5.2381 | ('alice', 'charlie')
2 | 5.4545 | ('bob', 'charlie')
2 | 3.6464 | ('alice', 'bob', 'charlie')
3 | 4.95 | ('alice', 'bob')
3 | 4.5 | ('alice', 'charlie')
3 | 4.95 | ('bob', 'charlie')
3 | 3.1935 | ('alice', 'bob', 'charlie')
4 | 5.7391 | ('alice', 'bob')
4 | 5.1429 | ('alice', 'charlie')
4 | 4.95 | ('bob', 'charlie')
4 | 3.5044 | ('alice', 'bob', 'charlie')


If we thereby choose to assign both alice, bob and charlie to the four tasks in the project, we may expect a duration of:

In [25]:
sum( [time for task, time, people in team if people == ('alice', 'bob', 'charlie')])

13.655418794934501

In [35]:
sum( [time for task, time, people in team if people == ('alice', 'bob') and task>1]) + 3.311

19.739260869565218

This number is important as this constitutes the lower bound.
Likewise the upper upperbound is the worst case assignment:

In [32]:
wc = []
for task_index, task in enumerate(tasks):
    max_value, p_max = 0, None
    for person, values in estimates.items():
        if values[task_index] > max_value:
            max_value = values[task_index]
            p_max = person
    wc.append( (task, p_max, max_value) )
    print(task, "|", p_max, "|", max_value)

print("worst case:", sum( [time for task, person, time in wc]) )

1 | bob | 11
2 | bob | 12
3 | bob | 11
4 | alice | 12
worst case: 46


Depending on how resources may be assigned to other projects, this is the range by which we have to operate. 

To great realisation, this also means that the critical path is thereby in the range from $[13.66; 46]$. I'm not sure many project management tools include this aspect. I certainly haven't seen this comparison before.

The next aspect is then to figure out how to assign resources across the programmes.
Let's for simplicity's sake assume that there are 2 programmes: A and B.

- A can start anytime, but must finish by time 20.
- B can't start before time 5, but has access restriction which means that task 3 must be done in the time interval $[20;25]$.

We know that Alice, Bob & Charlie can work on either project and that the most compressed horizon is with all three working full time on either project: $13.66 * 2 = 27.32$

If, for example Alice, Bob & Charlie all work on project A, until project B can be started, then the timeline becomes:

Project A

| task | start | end | team | comment |
|---|---|---|---|---|
| 1 | 0.000 | 3.311 | A,B,C | |
| 2 | 3.311 | 5.000 | A,B,C | Charlie leaves after 1.689 / 3.6464 units of work = 46.32% |
| 2 | 5.000 | 8.081 | A,B |  The remaining work is 100% - 46.32% = 53.68%<br>The A+B time is 5.7391, so 53.68% * 5.7391 = 3.081 |
| 3 | 8.081 | 13.031 | A,B ||
| 4 | 13.031 | 18.7701 | A,B ||


Project B

| task | start | end | team | comment |
|---|---|---|---|---|
| 1 | 5.000 | 14.000 | C | |
| 2 | 14.000 | 18.7701 | C | A,B joins at 18.7701 <br> C has done 4.7701 / 10 units of work = 47.7% |
| 2 | 18.7701 | 20.677 | A,B,C | A+B+C time is 3.6464 * (100% - 47.7%) = 1.907 |
| 3 | 20.677 | 23.871 | A,B,C |
| 4 | 23.871 | 27.375 | A,B,C | 

In this way all project constraints are met.


With this idea solidly settled, the next job is to determine the algorithm that can identify this solution automatically.

However as not all resources are infinitely divisible or commutative, we need to distingush these by decoupling our tasks and resources through an intermediary: 

![](artwork/project_management_resource_sets.png)

This allows us to express that Eric has a boolean commitment, e.g. either do or don't do the whole task, whilst Doris may be able to work on the task at integer steps, e.g. whole hours, whilst Alice, Bob and Charlie can shift at any timestep.

We also see that members of Resource Set 1 may be combined in any way to solve the task, and note the distinction that only Charlie and Eric are members of two resource sets.

The assignment problem is then to iterate through the resource sets to determine a combination of availability and resource type that is valid. This `subset` of all the possible combinations determine what we may refer to as a "valid assignment".

## Recap

Let's revist our observations for a second:

**Definitions**

- `Projects` and `Resources` belong to `Programmes` 
- `Projects` have `Tasks` 
- `Tasks` may depend on other `Tasks` or `Resources`
- `Tasks` have calendars that declare when they may be available for work.
- `Resources` have `Calendars` that declare their availability to do work.
- `ResourceSets` permit combinations of `Resources` so that tasks may be performed.

![](artwork/project_management_class_diagram.png)

**Problem definition**

The assignment problem is to assure that `Resource` assignments can fulfill `Tasks` with overlapping timewindows.

**Other aspects**

The project timeline is merely view of the collection of tasks.


## Time for some Python

Let's write some `classes`:

In [ ]:
from itertools import count

class Task(object):
    _ids = count()
    def __init__(self, description) -> None:
        assert isinstance(description, str)
        self.id = next(Task._ids)
        self.type = description
        self.deadline = None  # must be datetime
        self.fine = None  # must be function that takes a datetime
        self.duration = None  # must be timedelta or a callable function (CDF).
        self.earliest_start = None  # must be datetime
        self.latest_start = None  # must be datetime
        self.start = None  # must be datetime
        self.finish = None  # must be datetime
        
    def __hash__(self) -> int:
        return hash((self.type, self.id))
    
    def attach(self, *args):
        self.resource_sets.update(args)

    def determine_valid_assignments(self):
        pass  # iterate through resource sets to determine valid assignments within the resource sets.

In [51]:
class GenericTask(object):
    descriptions = set()
    def __init__(self, description, divisible=None, resource_sets=None) -> None:
        
        if description in GenericTask.descriptions:
            raise ValueError("description already in usage")
        GenericTask.descriptions.add(description)
        self.description = description

        if divisible is None:
            self.divisible = False
        elif isinstance(divisible, (float,int,bool)):
            self.divisible = divisible
        else:
            raise TypeError()
    
        self.resource_sets = set()
        if resource_sets is not None:
            self.attach(resource_sets)
    
    def attach(self, *args):
        self.resource_sets.update(args)

    def __call__(self) -> Task:
        return Task(self.description)
    
    def __hash__(self) -> int:
        return hash(self.description)

In [ ]:
from graph import Graph

class Project(object):
    _ids = count()
    def __init__(self) -> None:
        self._id = f"Project-{next(Project._ids)}"
        self.tasks = {}
        self.dependencies = Graph()

    def add_task(self):
        t = Task(project=self)
        self.tasks[t.id] = t
        return t

    def add_dependency(self, task1, task2):
        self.dependencies.add_edge(task1.id, task2.id)

    def critical_path(self):  # TODO
        pass # returns the critical path in the project
    
    def propagate(self, task):  # TODO
        pass # cascades the start time from `task` downstream.

In [ ]:
class Programme(object):
    def __init__(self, resources=None) -> None:
        self.projects = {}
        self.resources = set()
        if resources is not None:
            self.attach(resources)
    
    def attach(self, *resources):
        self.resources.update(set(resources))

    def new_project(self):
        p = Project()
        self.projects[p.id] = p
        return p

    def schedule():
        pass # finds a working schedule.

We also need to acknowledge that not all resources are available at all times. Alice may for example be away (holiday?) at a particular point in time.
This attaches a calendar to each resource (https://github.com/root-11/root-11.github.io/blob/master/content/business_calendar.ipynb)

In [ ]:
from datetime import datetime,date,time,timedelta

In [ ]:
class RangeError(IndexError):
    pass


class BusinessCalendar(object):
    def __init__(self, open_days=[0,1,2,3,4,5,6], open_from=time(0), open_to=time(23,59,59), 
                 holidays=None, 
                 valid_from=date(1970,1,1), valid_to=date(2060,1,1)) -> None:
        """
        open_days: integer monday=0 to sunday=6
        open_from: datetime.time 
        open_to: datetime.time
        holidays: list of dates
        valid_from: start datetime of calendar
        valid_to: end datetime of calendar
        """
        if not isinstance(open_days, list) and all(0<=i<=6 for i in open_days):
            raise TypeError("Expects open_days as a list of integers from 0 to 6.")
        self.open_days = open_days

        
        if not isinstance(valid_from, (date,datetime)):
            raise TypeError
        self.valid_from = valid_from

        if not isinstance(valid_to, (date,datetime)):
            raise TypeError
        self.valid_to = valid_to

        if not isinstance(open_from, time):
            raise TypeError
        self.open_from = open_from

        if not isinstance(open_to, time):
            raise TypeError
        self.open_to = open_to

        if not isinstance(holidays, list) and all(isinstance(i,date) for i in holidays):
            raise TypeError

        if holidays is None:
            holidays = []
        self.holidays = holidays  # consider using https://pypi.org/project/holidays/

    def __str__(self) -> str:
        return f"Calendar({self.open_days}, {self.open_from}-{self.open_to} excl. {self.holidays} within {self.valid_from}-{self.valid_to}"

    def is_open(self, now):
        if not isinstance(now, datetime):
            raise TypeError
        return all([
           now.weekday() in self.open_days,
           now.date() not in self.holidays,
           self.open_from <= now.time() < self.open_to,
           self.valid_from <= now.date() <= self.valid_to
        ])
    
    def opens_next(self, now):
        """ return datetime when calendar opens next"""
        if self.is_open(now):
            return now
        if not (self.valid_from <= now.date() < self.valid_to):
            raise RangeError("outside calendar limit.")
        
        new = datetime(now.year, now.month, now.day, self.open_from.hour, self.open_from.minute, self.open_from.second, self.open_from.microsecond)
        while True:
            if new.date() in self.holidays or new.weekday() not in self.open_days:
                new += timedelta(days=1)
            else:
                break
        if self.valid_from <= new.date() < self.valid_to:
            return new
        else:
            raise RangeError("Next opening date is outside calendar range")
    
    def closes_next(self, now):
        """ returns datetime when calendar closes """
        now = self.opens_next(now)
        return datetime(now.year, now.month, now.day, 
                        self.open_to.hour, self.open_to.minute, self.open_to.second, 
                        self.open_to.microsecond)



In [ ]:
class BusinessCalendars(object):
    def __init__(self, calendars=None) -> None:
        self.calendars = calendars if isinstance(calendars, list) and all(isinstance(i, BusinessCalendar) for i in calendars) else []
        self.calendars.sort(key=lambda x: x.valid_from)
    
    def __iadd__(self, other):
        if not isinstance(other, BusinessCalendar):
            raise TypeError
        self.calendars.append(other)
        self.calendars.sort(key=lambda x: x.valid_from)

    def is_open(self, now):
        return any(c.is_open(now) for c in self.calendars)
    
    def opens_next(self, now):
        opens = []
        for c in self.calendars:
            try:
                opens.append(c.open_next(now))
            except RangeError:
                pass
        if not opens:
            raise RangeError(f"{now} not in any Calendars range.")
        return max(opens)
    
    def closes_next(self, now):
        closes = []
        for c in self.calendars:
            try:
                closes.append(c.closes_next(now))
            except RangeError:
                pass
        if not closes:
            raise RangeError(f"{now} not in any Calendars range.")
        return min(closes)

    def finish(self, start, duration):
        """Calculates finish time.

        Args:
            start (datetime): start time
            duration (timedelta): duration

        Returns:
            datetime: finish time
        """
        if not isinstance(start, datetime):
            raise TypeError
        if not isinstance(duration, timedelta):
            raise TypeError

        _remaining_time = duration
        _start = self.opens_next(start)
        while _remaining_time > 0:
            _closes = self.closes_next(_start)
            dt = _closes - _start
            if _remaining_time > dt:
                _start = self.opens_next(_closes)
                _remaining_time -= dt
            else:
                return _start + _remaining_time


In [ ]:
summer = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(9), open_to=time(21), holidays=[], valid_from=date(2022,4,1), valid_to=date(2022,9,1))
winter = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(10), open_to=time(22), holidays=[], valid_from=date(2022,9,1), valid_to=date(2023,4,1))
calendars = BusinessCalendars(calendars=[summer,winter])

start = datetime(2022,8,27)
for ts in range(20):
    now = start+timedelta(days=ts)
    new = calendars.open_next(now)
    print(now, "open:", calendars.is_open(now), "-->", new, "open:", calendars.is_open(new), "in", (new-now).days, "days")

2022-08-27 00:00:00 open: False --> 2022-08-29 09:00:00 open: True in 2 days
2022-08-28 00:00:00 open: False --> 2022-08-29 09:00:00 open: True in 1 days
2022-08-29 00:00:00 open: False --> 2022-08-29 09:00:00 open: True in 0 days
2022-08-30 00:00:00 open: False --> 2022-08-30 09:00:00 open: True in 0 days
2022-08-31 00:00:00 open: False --> 2022-08-31 09:00:00 open: True in 0 days
2022-09-01 00:00:00 open: False --> 2022-09-01 10:00:00 open: True in 0 days
2022-09-02 00:00:00 open: False --> 2022-09-02 10:00:00 open: True in 0 days
2022-09-03 00:00:00 open: False --> 2022-09-05 10:00:00 open: True in 2 days
2022-09-04 00:00:00 open: False --> 2022-09-05 10:00:00 open: True in 1 days
2022-09-05 00:00:00 open: False --> 2022-09-05 10:00:00 open: True in 0 days
2022-09-06 00:00:00 open: False --> 2022-09-06 10:00:00 open: True in 0 days
2022-09-07 00:00:00 open: False --> 2022-09-07 10:00:00 open: True in 0 days
2022-09-08 00:00:00 open: False --> 2022-09-08 10:00:00 open: True in 0 days

Will 3 shifts of 7 hours work too?

|shift| start | end |
|---|---|---|
|1|0800|1500|
|2|1600|2300|
|3|0000|0700|

1 hour maintenance shutdown between shifts.

In [ ]:
shift_1 = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(8), open_to=time(15), holidays=[], valid_from=date(2022,4,1), valid_to=date(2022,9,1))
shift_2 = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(16), open_to=time(23), holidays=[], valid_from=date(2022,4,1), valid_to=date(2022,9,1))
shift_3 = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(0), open_to=time(7), holidays=[], valid_from=date(2022,4,1), valid_to=date(2022,9,1))

calendar = BusinessCalendars([shift_1,shift_2,shift_3])

In [ ]:
now = datetime(2022,4,21,6,10)
for h in range(24):
    now += timedelta(hours=1)
    print(now, "is open:", calendar.is_open(now))

2022-04-21 07:10:00 is open: False
2022-04-21 08:10:00 is open: True
2022-04-21 09:10:00 is open: True
2022-04-21 10:10:00 is open: True
2022-04-21 11:10:00 is open: True
2022-04-21 12:10:00 is open: True
2022-04-21 13:10:00 is open: True
2022-04-21 14:10:00 is open: True
2022-04-21 15:10:00 is open: False
2022-04-21 16:10:00 is open: True
2022-04-21 17:10:00 is open: True
2022-04-21 18:10:00 is open: True
2022-04-21 19:10:00 is open: True
2022-04-21 20:10:00 is open: True
2022-04-21 21:10:00 is open: True
2022-04-21 22:10:00 is open: True
2022-04-21 23:10:00 is open: False
2022-04-22 00:10:00 is open: True
2022-04-22 01:10:00 is open: True
2022-04-22 02:10:00 is open: True
2022-04-22 03:10:00 is open: True
2022-04-22 04:10:00 is open: True
2022-04-22 05:10:00 is open: True
2022-04-22 06:10:00 is open: True


Let's test the idea that we want to calculate when something will finish, given the inputs:
- a calendar, 
- a point in time 
- a duration.

In [ ]:
cal = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(8), open_to=time(16))
start=datetime(2022,4,21,0,0,0,0)
duration=timedelta(hours=100)
finish = cal.finish(start, duration)
finish

In [ ]:
def intercept(c1,c2):  # TODO
    pass  # calculate the intercept of two calendars as a new calendar.

### How to get official holidays

To get holiday calendars worldwide base tasks on: `pip install workalendar`

```python
>>> from datetime import date
>>> from workalendar.europe import France
>>> cal = France()
>>> cal.holidays(2012)
[(datetime.date(2012, 1, 1), 'New year'),
 (datetime.date(2012, 4, 9), 'Easter Monday'),
 (datetime.date(2012, 5, 1), 'Labour Day'),
 (datetime.date(2012, 5, 8), 'Victory in Europe Day'),
 (datetime.date(2012, 5, 17), 'Ascension Day'),
 (datetime.date(2012, 5, 28), 'Whit Monday'),
 (datetime.date(2012, 7, 14), 'Bastille Day'),
 (datetime.date(2012, 8, 15), 'Assumption of Mary to Heaven'),
 (datetime.date(2012, 11, 1), "All Saints' Day"),
 (datetime.date(2012, 11, 11), 'Armistice Day'),
 (datetime.date(2012, 12, 25), 'Christmas')]
```

You can get add them using:

```python
>>> year = 2023
>>> cal = BusinessCalendar(holidays=cal.holidays(2023), valid_from=date(2023,1,1), valid_to=date(2023,12,31)
```


In [ ]:
class Resource(object):
    def __init__(self, divisble=None) -> None:
        if divisble is None:
            self.divisible = False
        elif isinstance(divisble, (int, float, bool)):
            self.divisible = divisble
        else:
            raise TypeError
        self.availability = BusinessCalendars()
        self.estimates = {}
        self.schedule = BusinessCalendars()  # a calendar with occupied intervels.

In [ ]:
class ResourceSet(object):
    def __init__(self, **kwargs) -> None:        
        self.resources = {}

        for resource, duration in kwargs.items():
            if isinstance(resource, Resource):
                self.resources[resource] = duration
                self.resources[(resource, )] = duration
            elif isinstance(resource, tuple):
                self.resources[resource] = duration
            else:
                raise TypeError()
                    
    def add(self, duration, resource):
        if isinstance(resource, Resource):
            pass
        elif isinstance(resource, tuple):
            if all(isinstance(i, Resource) for i in tuple):
                pass
        else:
            raise TypeError()
        
        self.resources[resource] = duration
    
    def __getitem__(self, item):
        return self.resources.get(item, None)

    def __iter__(self):
        return self.resources[:]


Let's add a small test...

In [ ]:
# First create resources
alice = Resource(divisble=True)  # has default calendar.
bob = Resource(divisble=True)
charlie = Resource(divisble=True)
doris = Resource(divisble=1.0)
eric = Resource()  # defaults to divisible = False

all_resources = [alice, bob, charlie, doris, eric]

# Then create resource sets and add singular resource estimates
rs1 = ResourceSet(**{alice:10, bob:11, charlie:9})  # resource set for task 1.
rs2 = ResourceSet(**{alice:11, bob:12, charlie:10})
rs3 = ResourceSet(**{alice:9,  bob:11, charlie:9})
rs4 = ResourceSet(**{alice:12, bob:11, charlie:9})

# doris and eric don't have individual estimates. Only joint estimates:
rs5 = ResourceSet(**{(doris, eric): 12.5})
rs6 = ResourceSet(**{(charlie, eric): 12.9})  # charlie is leader
rs6 = ResourceSet(**{(eric, charlie): 13.1})  # eric is leader

# Add joint resource estimates for alice, bob and charlie:
for rg in [rs1,rs2,rs3,rs4]:
    for combination in powerset([alice,bob,charlie],min_=2):  # (alice, bob)  --> ([10,11,9,12], [11,12,11,11])
        times = [ rg[person] for person in combination ]
        time = inv_mean(times)
        rg.add(time, combination)

# Then create generic tasks and link them with resource groups.
gt1 = GenericTask('1', divisible=0.001, resource_sets={rs1, rs5})
gt2 = GenericTask('2', divisible=0.001, resource_sets={rs2, rs5})
gt3 = GenericTask('3', divisible=0.001, resource_sets={rs3, rs6})
gt4 = GenericTask('4', divisible=0.001, resource_sets={rs4, rs6})

# As the baseline for resource estimates are in place, the next task is to 
# create the projects with task instances.

programme = Programme(resources=all_resources)  # create programme and attach resources
project_1 = programme.new_project()
project_2 = programme.new_project()

for project in [project_1, project_2]:
    last_task = None
    for generic_task in [gt1, gt2, gt3, gt4]:
        project_task = generic_task()  # this task is an independent copy of the generic task
        project_1.add_task(project_task)
        if last_task is not None:
            project.add_dependency(last_task,project_task)
        else:
            last_task = project_task
        
# now the programme contains two projects and each project has 4 tasks, but the 
# constraints are missing:
p1t4 = [t for t in project_1.tasks.values() if t.type == '4'][0]
p1t4.finish = 20  # project 1 must finish at time 20.

p2t3 = [t for t in project_2.tasks.values() if t.type == '3'][0]
p2t3.earliest_start = 20  # project 2 task 3 can only happen in t=[20;25]
p2t3.latest_finish = 25

# TODO:
# ------
programme.schedule()  # find a working schedule

for resource in programme.resources:
    for task in resource.task_list:
        print(resource, "|", task)


With this outline the next step is to schedule the resources in such a way that all work is leveled as far as possible.